# Workplace for: Customer: 14; DFU: Рис басмати 500 гр

In [41]:
import plotly.express as px
from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from xgboost import XGBClassifier, XGBRegressor

In [42]:
df = pd.read_csv("data_grouped/train_group_14_Рис басмати 500 гр.csv")
sales_2_df_t = pd.read_csv("data_test_grouped/test_group_14_Рис басмати 500 гр.csv")
df, sales_2_df_t

(                    DFU  Customer      Period  BPV  Total Sell-in  Season  \
 0    Рис басмати 500 гр        14  2022-01-24  NaN            NaN  Winter   
 1    Рис басмати 500 гр        14  2022-01-31  NaN            NaN  Winter   
 2    Рис басмати 500 гр        14  2022-02-07  NaN            NaN  Winter   
 3    Рис басмати 500 гр        14  2022-02-14  NaN            NaN  Winter   
 4    Рис басмати 500 гр        14  2022-02-21  NaN            NaN  Winter   
 ..                  ...       ...         ...  ...            ...     ...   
 240  Рис басмати 500 гр        14  2021-12-20  NaN            NaN  Winter   
 241  Рис басмати 500 гр        14  2021-12-27  NaN            NaN  Winter   
 242  Рис басмати 500 гр        14  2022-01-03  NaN            NaN  Winter   
 243  Рис басмати 500 гр        14  2022-01-10  NaN            NaN  Winter   
 244  Рис басмати 500 гр        14  2022-01-17  NaN            NaN  Winter   
 
             Type Geography End of Period  BPV_sale_period  So

## Visualization

In [43]:
unique_stat_colors = {
    'BPV Mean': '#1f77b4',
    'BPV Median': '#ff7f0e',
    'BPV CI Low (95%)': '#2ca02c',
    'BPV CI Up (95%)': '#d62728',
    'Total Sell-in Mean': '#9467bd',
    'Total Sell-in Median': '#8c564b',
    'Total Sell-in CI Low (95%)': '#e377c2',
    'Total Sell-in CI Up (95%)': '#7f7f7f',
}

for (dfu, customer), group in df.groupby(['DFU', 'Customer']):
    melted_group = group.melt(id_vars=['Period'], value_vars=['BPV', 'Total Sell-in'],
                              var_name='Metric', value_name='Value')

    stat_lines = []

    for metric in ['BPV', 'Total Sell-in']:
        data = group[metric].dropna()
        if not data.empty:
            mean_val = data.mean()
            median_val = data.median()
            std_val = data.std()

            stat_lines.extend([
                (metric, f'{metric} Mean: {mean_val:.2f}', mean_val, unique_stat_colors[f'{metric} Mean']),
                (metric, f'{metric} Median: {median_val:.2f}', median_val, unique_stat_colors[f'{metric} Median'])
            ])

    fig = px.line(
        melted_group,
        x='Period', y='Value', color='Metric',
        title=f'DFU: {dfu} | Customer: {customer}',
        labels={'Value': 'Sales', 'Period': 'Date'}
    )

    for metric, label, value, color in stat_lines:
        fig.add_trace(go.Scatter(
            x=[group['Period'].min(), group['Period'].max()],
            y=[value, value],
            mode='lines',
            name=label,
            line=dict(dash='dot', color=color, width=1.5),
            showlegend=True
        ))

    for metric in ['BPV', 'Total Sell-in']:
        metric_df = group[['Period', metric]].dropna()
        if not metric_df.empty:
            metric_df = metric_df.copy()
            metric_df['Period_ordinal'] = pd.to_datetime(metric_df['Period']).map(pd.Timestamp.toordinal)
            X = metric_df[['Period_ordinal']]
            y = metric_df[metric]
            model = LinearRegression().fit(X, y)
            y_pred = model.predict(X)

            fig.add_trace(go.Scatter(
                x=metric_df['Period'],
                y=y_pred,
                mode='lines',
                name=f'Trend - {metric}',
                line=dict(dash='dash', color='orange')
            ))

    if group[['BPV', 'Total Sell-in']].dropna().shape[0] > 1:
        corr = group['BPV'].corr(group['Total Sell-in'])
        fig.add_annotation(
            text=f"Корреляция BPV и Total Sell-in: {corr:.2f}",
            xref="paper", yref="paper",
            x=0.99, y=1.05, showarrow=False,
            font=dict(size=12, color="white"),
            align="right",
            bordercolor="white", borderwidth=1
        )

    filtered_sales_2 = sales_2_df_t[(sales_2_df_t['DFU'] == dfu) & (sales_2_df_t['Customer'] == customer)]
    if not filtered_sales_2.empty:
        melted_sales_2 = filtered_sales_2.melt(id_vars=['Period'], value_vars=['BPV', 'Total Sell-in'],
                                               var_name='Metric', value_name='Value')
        for metric in melted_sales_2['Metric'].unique():
            metric_data = melted_sales_2[melted_sales_2['Metric'] == metric]
            color = 'black' if metric == 'BPV' else 'gray'
            fig.add_trace(go.Scatter(
                x=metric_data['Period'],
                y=metric_data['Value'],
                mode='lines+markers',
                name=f'test_sales - {metric}',
                line=dict(color=color, width=2, dash='dot')
            ))

    fig.update_xaxes(title_text='Дата', tickformat='%Y-%m-%d')
    fig.update_yaxes(title_text='Значение')
    fig.show()


## Machine learning

In [44]:
def compute_metrics(y_true, y_pred, m=1):
    """
    Compute forecast accuracy metrics: WAPE, MAE, RMSE, MASE.

    Parameters
    ----------
    y_true : array-like of shape (n,)
        Истинные значения.
    y_pred : array-like of shape (n,)
        Предсказанные значения.
    m : int, default=1
        Шаг «наивного» прогноза для расчёта MASE.
        Для несезонного ряда обычно m=1;
        для сезонного — период сезонности.

    Returns
    -------
    dict
        {
            'WAPE': float,  # в процентах
            'MAE': float,
            'RMSE': float,
            'MASE': float
        }
    """
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    n = len(y_true)

    # WAPE
    num = np.abs(y_true - y_pred).sum()
    denom = np.abs(y_true).sum()
    wape = num / denom * 100 if denom != 0 else np.nan

    # MAE
    mae = np.mean(np.abs(y_true - y_pred))

    # RMSE
    rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))

    # MASE
    if n > m:
        naive_errors = np.abs(y_true[m:] - y_true[:-m])
        scale = np.mean(naive_errors)
        mase = mae / scale if scale != 0 else np.nan
    else:
        mase = np.nan

    return {
        'WAPE': wape,
        'MAE': mae,
        'RMSE': rmse,
        'MASE': mase
    }

### Загрузка данных для группы 14 — "Рис басмати 500 гр"

- Загружаются обучающая и тестовая выборки, соответствующие 14-й группе товара.
- Обучающая выборка фильтруется по дате: остаются записи с 23 октября 2020 года по 17 января 2022 года.
- В тестовой выборке столбец `BPV` переименовывается в `BPV_true` для единообразия и последующего сравнения предсказаний с фактами.

In [45]:
train_path = "data_grouped/train_group_14_Рис басмати 500 гр.csv"
test_path = "data_test_grouped/test_group_14_Рис басмати 500 гр.csv"

train = pd.read_csv(train_path)
train = train[(train["Period"] >= "2020-10-23") & (train["Period"] <= "2022-01-17")].copy()
test = pd.read_csv(test_path).rename(columns={"BPV": "BPV_true"})

### Функция `enrich`: генерация календарных и сезонных признаков

Функция добавляет к датафрейму `df` следующие признаки, полезные для учёта временных и сезонных эффектов:

- `Period` — преобразование даты в формат `datetime`.
- `Period_ord` — числовое представление даты (количество дней от начала эпохи).
- `weekofyear` — номер недели в году.
- `month`, `day`, `days_in_month` — календарные признаки, включая количество дней в месяце.
- `week_share` — доля месяца, пройденная к текущей дате (`day / days_in_month`).
- `sin_month`, `cos_month` — синус и косинус месяца, используются для моделирования сезонности в циклическом формате.
- `Period_last_year` — дата, соответствующая аналогичному периоду год назад (смещение на 52 недели назад).

In [46]:
def enrich(df):
    df["Period"] = pd.to_datetime(df["Period"])
    df["Period_ord"] = df["Period"].map(pd.Timestamp.toordinal)
    df["weekofyear"] = df["Period"].dt.isocalendar().week.astype(int)
    df["month"] = df["Period"].dt.month
    df["day"] = df["Period"].dt.day
    df["days_in_month"] = df["Period"].dt.days_in_month
    df["week_share"] = df["day"] / df["days_in_month"]
    df["sin_month"] = np.sin(2 * np.pi * df["month"] / 12)
    df["cos_month"] = np.cos(2 * np.pi * df["month"] / 12)
    df["Period_last_year"] = df["Period"] - pd.DateOffset(weeks=52)
    return df

### Обогащение обучающей и тестовой выборок временными признаками

Функция `enrich` применяется к датафреймам `train` и `test` для добавления временных и сезонных признаков.
Это необходимо для более точного моделирования закономерностей, связанных с календарём и сезонностью продаж.

In [47]:
train = enrich(train)
test = enrich(test)

### Добавление признака продаж за аналогичный период прошлого года

- Создаётся таблица `sellin_map`, содержащая даты и значения `Total Sell-in`, переименованные в `Period_last_year` и `sellin_last_year`.
- Выполняется объединение `train` и `test` с этой таблицей по колонке `Period_last_year`, чтобы добавить информацию о продажах в ту же неделю год назад.
- Пропущенные значения заполняются нулями, если в прошлом году данных за данный период не было.
- Логарифмирование признака `sellin_last_year` с помощью `np.log1p` (логарифм от `1 + x`) позволяет сгладить влияние больших значений и выбросов.

In [48]:
sellin_map = train[["Period", "Total Sell-in"]].rename(
    columns={"Period": "Period_last_year", "Total Sell-in": "sellin_last_year"}
)
train = train.merge(sellin_map, on="Period_last_year", how="left")
test = test.merge(sellin_map, on="Period_last_year", how="left")
train["sellin_last_year"].fillna(0, inplace=True)
test["sellin_last_year"].fillna(0, inplace=True)

train["log_sellin_last_year"] = np.log1p(train["sellin_last_year"])
test["log_sellin_last_year"] = np.log1p(test["sellin_last_year"])

/var/folders/lm/kymf9kqj2kbfvq9cx_8cpz5h0000gn/T/ipykernel_39009/2863312752.py:6: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.



/var/folders/lm/kymf9kqj2kbfvq9cx_8cpz5h0000gn/T/ipykernel_39009/2863312752.py:7: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing '

### Подготовка признаков и целевых переменных для классификации и регрессии

- `feat_cols` — список признаков, включающий временные характеристики, отложенные продажи и сезонные преобразования (в том числе синус и косинус месяца).
- `train_clf` — обучающая выборка, содержащая только строки с ненулевыми значениями `BPV`.
- `sale_flag` — бинарная переменная, обозначающая факт наличия продаж (`1`, если `BPV > 0`, иначе `0`).
- `X_clf`, `y_clf` — признаки и целевая переменная для задачи классификации (предсказание факта продажи).
- `train_reg` — подмножество обучающей выборки, содержащее только периоды с фактическими продажами.
- `X_reg`, `y_reg` — признаки и логарифмированная целевая переменная для регрессии (предсказание объёма продаж).
- `X_test` — признаки из тестовой выборки.
- `true_test` — фактические значения продаж из тестовой выборки, используемые для последующей оценки модели.

In [49]:
feat_cols = [
    "Period_ord", "weekofyear", "month",
    "sellin_last_year", "log_sellin_last_year",
    "week_share", "sin_month", "cos_month"
]

train_clf = train[train["BPV"].notna()].copy()
train_clf["sale_flag"] = (train_clf["BPV"] > 0).astype(int)

X_clf = train_clf[feat_cols]
y_clf = train_clf["sale_flag"]

train_reg = train_clf[train_clf["sale_flag"] == 1].copy()
X_reg = train_reg[feat_cols]
y_reg = np.log1p(train_reg["BPV"])

X_test = test[feat_cols].reset_index(drop=True)
true_test = test[["Period", "BPV_true"]].reset_index(drop=True)

### Обучение ансамбля моделей: классификаторов и регрессоров (XGBoost)

Цикл по `N_MODELS = 5` с различными `random_state` обучает ансамбль из 5 пар моделей:

1. **Классификатор (`XGBClassifier`)**:
   - Предсказывает вероятность того, что в периоде произойдут продажи (`sale_flag = 1`).
   - Использует `scale_pos_weight` для компенсации дисбаланса классов (большинство примеров — нулевые продажи).
   - Предсказанные вероятности наличия продаж (`proba_test`) сохраняются в список `probas`.

2. **Регрессор (`XGBRegressor`)**:
   - Предсказывает объём продаж (`BPV`) только для примеров с ненулевыми продажами.
   - Модель обучается на логарифмированных значениях `BPV` для сглаживания распределения.
   - После предсказания значения преобразуются обратно из логарифмической шкалы с помощью `expm1`.
   - Все отрицательные значения в прогнозе обнуляются.
   - Предсказания сохраняются в список `preds`.

Итог — ансамбль из 5 классификаторов и 5 регрессоров, чьи результаты далее будут агрегированы.

In [50]:
N_MODELS = 5
probas = []
preds = []

for seed in range(42, 42 + N_MODELS):
    clf = XGBClassifier(
        n_estimators=500,
        learning_rate=0.1,
        max_depth=5,
        scale_pos_weight=(y_clf == 0).sum() / (y_clf == 1).sum(),
        use_label_encoder=False,
        eval_metric="logloss",
        random_state=seed,
        verbosity=0
    )
    clf.fit(X_clf, y_clf)
    proba_test = clf.predict_proba(X_test)[:, 1]
    probas.append(proba_test)

    reg = XGBRegressor(
        n_estimators=600,
        learning_rate=0.03,
        max_depth=5,
        random_state=seed,
        verbosity=0
    )
    reg.fit(X_reg, y_reg)
    pred = np.expm1(reg.predict(X_test))
    preds.append(np.maximum(pred, 0))

### Агрегация предсказаний ансамбля и оценка финальных результатов

- `proba_test_median` — медианная вероятность наличия продаж по результатам всех классификаторов.
- `base_pred_median` — медианный прогноз объёма продаж по результатам всех регрессоров.
- `threshold = 0.1` — порог вероятности, выше которого считается, что продажа произойдёт.
- `is_sale` — бинарный массив, в котором `True` соответствует периодам, где модель ожидает наличие продаж.
- `y_pred` — финальный прогноз:
  - если `is_sale = True`, берётся медианный прогноз объёма продаж;
  - иначе — прогноз равен 0.

- `merged` — итоговая таблица с датами (`Period`), реальными значениями (`BPV_true`) и предсказаниями (`BPV_pred`).

- Вычисление метрик:
  - `WAPE` — взвешенная абсолютная ошибка в процентах,
  - `MAE` — средняя абсолютная ошибка,
  - `RMSE` — среднеквадратичная ошибка,
  - `MASE` — масштабированная ошибка (при `m=1`, предполагается недельная сезонность).

Метрики печатаются в консоль и позволяют оценить качество прогноза ансамбля.

In [51]:
proba_test_median = np.median(probas, axis=0)
base_pred_median = np.median(preds, axis=0)

threshold = 0.1
is_sale = proba_test_median >= threshold
y_pred = np.where(is_sale, base_pred_median, 0)

merged = pd.DataFrame({
    "Period": true_test["Period"],
    "BPV_true": true_test["BPV_true"],
    "BPV_pred": y_pred
})

metrics = compute_metrics(merged["BPV_true"], merged["BPV_pred"], m=1)

print(f"WAPE = {metrics['WAPE']:.2f}%")
print(f"MAE  = {metrics['MAE']:.4f}")
print(f"RMSE = {metrics['RMSE']:.4f}")
print(f"MASE = {metrics['MASE']:.4f}")

WAPE = 53.61%
MAE  = 3.1713
RMSE = 4.5139
MASE = 0.6657


In [52]:
to_save = merged.drop(columns=["BPV_true"])
to_save.to_csv("14_Рис басмати 500_гр_eval_predictions.csv", index=False)

In [53]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=merged["Period"], y=merged["BPV_true"],
    mode="markers+lines",
    name="BPV_true",
    line=dict(color="blue"),
    marker=dict(size=6)
))
fig.add_trace(go.Scatter(
    x=merged["Period"], y=merged["BPV_pred"],
    mode="markers+lines",
    name="BPV_pred",
    line=dict(color="red", dash="dot"),
    marker=dict(size=6)
))
fig.update_layout(
    title="14 Рис басмати 500 гр. Сравнение BPV_true и BPV_pred",
    xaxis_title="Period",
    yaxis_title="BPV",
    legend_title="Легенда"
)
fig.show()

Prediction for 1.5 years using previous pipeline

In [54]:
train_path = "data_grouped/train_group_14_Рис басмати 500 гр.csv"

train = pd.read_csv(train_path)
train = train[(train["Period"] >= "2020-10-23")].copy()
test = train[train["BPV"].isna()].copy()

In [55]:
def enrich(df):
    df["Period"] = pd.to_datetime(df["Period"])
    df["Period_ord"] = df["Period"].map(pd.Timestamp.toordinal)
    df["weekofyear"] = df["Period"].dt.isocalendar().week.astype(int)
    df["month"] = df["Period"].dt.month
    df["day"] = df["Period"].dt.day
    df["days_in_month"] = df["Period"].dt.days_in_month
    df["week_share"] = df["day"] / df["days_in_month"]
    df["sin_month"] = np.sin(2 * np.pi * df["month"] / 12)
    df["cos_month"] = np.cos(2 * np.pi * df["month"] / 12)
    df["Period_last_year"] = df["Period"] - pd.DateOffset(weeks=52)
    return df

In [56]:
train = enrich(train)
test = enrich(test)

In [57]:
sellin_map = train[["Period", "Total Sell-in"]].rename(
    columns={"Period": "Period_last_year", "Total Sell-in": "sellin_last_year"}
)
train = train.merge(sellin_map, on="Period_last_year", how="left")
test = test.merge(sellin_map, on="Period_last_year", how="left")
train["sellin_last_year"].fillna(0, inplace=True)
test["sellin_last_year"].fillna(0, inplace=True)

train["log_sellin_last_year"] = np.log1p(train["sellin_last_year"])
test["log_sellin_last_year"] = np.log1p(test["sellin_last_year"])

/var/folders/lm/kymf9kqj2kbfvq9cx_8cpz5h0000gn/T/ipykernel_39009/2863312752.py:6: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.



/var/folders/lm/kymf9kqj2kbfvq9cx_8cpz5h0000gn/T/ipykernel_39009/2863312752.py:7: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing '

In [58]:
feat_cols = [
    "Period_ord", "weekofyear", "month",
    "sellin_last_year", "log_sellin_last_year",
    "week_share", "sin_month", "cos_month"
]

train_clf = train[train["BPV"].notna()].copy()
train_clf["sale_flag"] = (train_clf["BPV"] > 0).astype(int)

X_clf = train_clf[feat_cols]
y_clf = train_clf["sale_flag"]

train_reg = train_clf[train_clf["sale_flag"] == 1].copy()
X_reg = train_reg[feat_cols]
y_reg = np.log1p(train_reg["BPV"])

X_test = test[feat_cols].reset_index(drop=True)

In [59]:
N_MODELS = 5
probas = []
preds = []

for seed in range(42, 42 + N_MODELS):
    clf = XGBClassifier(
        n_estimators=500,
        learning_rate=0.1,
        max_depth=5,
        scale_pos_weight=(y_clf == 0).sum() / (y_clf == 1).sum(),
        use_label_encoder=False,
        eval_metric="logloss",
        random_state=seed,
        verbosity=0
    )
    clf.fit(X_clf, y_clf)
    proba_test = clf.predict_proba(X_test)[:, 1]
    probas.append(proba_test)

    reg = XGBRegressor(
        n_estimators=600,
        learning_rate=0.03,
        max_depth=5,
        random_state=seed,
        verbosity=0
    )
    reg.fit(X_reg, y_reg)
    pred = np.expm1(reg.predict(X_test))
    preds.append(np.maximum(pred, 0))

In [60]:
proba_test_median = np.median(probas, axis=0)
base_pred_median = np.median(preds, axis=0)

threshold = 0.1
is_sale = proba_test_median >= threshold
y_pred = np.where(is_sale, base_pred_median, 0)

result = pd.DataFrame({
    "Period": test["Period"],
    "BPV_pred": y_pred
})

In [61]:
result.to_csv("14_Рис басмати 500 гр_predictions.csv", index=False)